In [ ]:
import pandas as pd
from currency_converter import CurrencyConverter
from sklearn.preprocessing import MultiLabelBinarizer


In [ ]:
df = pd.read_csv('Data/survey_results_public.csv') #not included in the repo

In [ ]:
df.head()

In [ ]:
df.Country.value_counts().head(n=20).plot.bar()

# DataPreprocessing

In [ ]:
# remove nan's
new = df[['Employment', 'RemoteWork', 'EdLevel', 'YearsCode', 'Country', 'OrgSize', 'Currency', 'CompTotal', 'CompFreq', 'LanguageHaveWorkedWith']].copy()
new.info()
new_df = new.dropna(subset=['Employment', 'Country', 'CompTotal', 'CompFreq', 'LanguageHaveWorkedWith'])

### Employment

In [ ]:
new_df['Employment'] = new_df['Employment'].replace(to_replace=r'^Employed.*$', value='Employed', regex=True)
new_df['Employment'] = new_df['Employment'].replace(to_replace=r'^Independent.*$', value='Independend', regex=True)
new_df.Employment.value_counts()

In [ ]:
new_df.rename(columns={'Employment': "Employed"}, inplace=True)

mapping = {'Employed': 1, 'Independend': 0}
new_df['Employed'] = new_df['Employed'].map(mapping)
new_df.Employed.value_counts()

### Preprocess: RemoteWork

In [ ]:
#RemoteWork
new_df['RemoteWork'] = new_df['RemoteWork'].replace(to_replace=r'^Fully remote.*$', value='remote', regex=True)
new_df['RemoteWork'] = new_df['RemoteWork'].replace(to_replace=r'^Hybrid.*$', value='hybrid', regex=True)
new_df['RemoteWork'] = new_df['RemoteWork'].replace(to_replace=r'^Full in-person.*$', value='office', regex=True)

new_df.RemoteWork.value_counts()

### EdLevel

In [ ]:
new_df.EdLevel.value_counts()

In [ ]:
mapping = {
'Other doctoral degree (Ph.D., Ed.D., etc.)': 8,
'Master’s degree (M.A., M.S., M.Eng., MBA, etc.)': 7,
'Bachelor’s degree (B.A., B.S., B.Eng., etc.)': 6,
'Associate degree (A.A., A.S., etc.)': 5,
'Some college/university study without earning a degree': 4,
'Professional degree (JD, MD, etc.)': 3,
'Secondary school (e.g. American high school, German Realschule or Gymnasium, etc.)': 2,
'Primary/elementary school': 1,
'Something else': 0
}
new_df['EdLevel'] = new_df['EdLevel'].map(mapping)
new_df['EdLevel'].value_counts()

### Organization Size

In [ ]:
mapping = {
    '10,000 or more employees': 9,
    '5,000 to 9,999 employees': 8,
    '1,000 to 4,999 employees': 7,
    '500 to 999 employees': 6,
    '100 to 499 employees': 5,
    '20 to 99 employees': 4,
    '10 to 19 employees': 3,
    '2 to 9 employees': 2,
    'Just me - I am a freelancer, sole proprietor, etc.': 1,
    'I don’t know': 0
}
new_df['OrgSize'] = new_df['OrgSize'].map(mapping)
new_df['OrgSize'].value_counts()

## Compensation

In [ ]:
new_df.Currency.value_counts()

In [ ]:
mapping = {
    'Yearly': 1,
    'Monthly': 12,
    'Weekly': 52
}
new_df['CompFreq'] = new_df['CompFreq'].map(mapping)
new_df['CompFreq'].value_counts()

In [ ]:
c = CurrencyConverter("Data/eurofxref-hist.csv")
new_df['CompYearEur'] = 0

currencies_not_supported = []
for index, row in new_df.iterrows():
    currency = row.Currency[:3]
    try:
        compensation_eur = c.convert(row.CompTotal, currency, 'EUR') * row.CompFreq
    except ValueError as e:
        currencies_not_supported.append(currency)
        compensation_eur = c.convert(row.CompTotal, 'USD', 'EUR') * row.CompFreq
    new_df.at[index, 'CompYearEur'] = compensation_eur

new_df = new_df[new_df["CompYearEur"] < 1000000]

new_df = new_df.round({"CompYearEur":0})

print(new_df)


In [ ]:
new_df.drop('Currency', axis=1, inplace=True)
new_df.drop('CompTotal', axis=1, inplace=True)
new_df.drop('CompFreq', axis=1, inplace=True)

### Countries

In [ ]:
mapping = {
    'United Kingdom of Great Britain and Northern Ireland': "United Kingdom",
    'Czech Republic': 'Czechia',
    'Russian Federation': 'Russia',
    'Viet Nam': 'Vietnam',
    'Hong Kong (S.A.R.)': 'Hong Kong',
    'Bosnia and Herzegovina': 'Bosnia and Herz.',
    'Iran, Islamic Republic of...': 'Iran',
    'Republic of Korea': 'North Korea',
    'The former Yugoslav Republic of Macedonia': 'Macedonia',
    'Syrian Arab Republic': 'Syria',
    'Republic of Moldova': 'Moldova',
    'United Republic of Tanzania': 'Tanzania',
    'Dominican Republic': 'Dominican Rep.',
    'Venezuela, Bolivarian Republic of...': 'Venezuela',
    "Lao People's Democratic Republic": 'Laos',
    'Democratic Republic of the Congo': 'Dem. Rep. Congo',
    'Congo, Republic of the...': 'Congo',
    'Libyan Arab Jamahiriya': 'Libya',
    'Cape Verde': 'Cabo Verde'
}
new_df['Country'] = new_df['Country'].replace(mapping)
new_df['Country'].value_counts()


In [ ]:
new_df.Country.value_counts()

### ProgrammingLanguages

In [ ]:
categories = {}

new_df['LanguageHaveWorkedWith'] = new_df['LanguageHaveWorkedWith'].str.split(';')

mlb = MultiLabelBinarizer()
one_hot_encoded = pd.DataFrame(mlb.fit_transform(new_df['LanguageHaveWorkedWith']),
                               columns=mlb.classes_,
                               index=new_df.index)

one_hot_encoded.rename(columns={col: '#' + col for col in one_hot_encoded.columns}, inplace=True)

new_df = pd.concat([new_df, one_hot_encoded], axis=1)

new_df.drop(["LanguageHaveWorkedWith"], axis=1, inplace=True)


### Export

In [ ]:
new_df.to_csv('Data/surveydata_v4.csv', index=False)